In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-14 14:03:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.66MB/s    in 0.6s    

2022-03-14 14:03:34 (1.66 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         DE|      10133| RVOG49N0H1FB6|B004TACMZ8|     569741360|Bosch GMS120 Ortu...|    Home Improvement|          5|            0|          0|   N|                Y|               Super|Delivery took a l...| 2014-08-01|
|         DE|      19612| RNCMD6OLTP4HM|1846071224|     785505948|The Wheels On The...|               Bo

In [5]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [6]:
type(df)

pyspark.sql.dataframe.DataFrame

### Create DataFrames to match tables

In [9]:
from pyspark.sql.functions import to_date, count
# Read in the Review dataset as a DataFrame


In [10]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")

In [13]:
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      38153|             3|
|    2094984|             1|
|    3006039|             1|
|    3121833|             1|
|    3391089|             8|
|    3470216|             2|
|    3529901|             1|
|    3744024|             1|
|    3768097|             1|
|    3804964|             1|
|    3908493|             1|
|    3949894|             1|
|    3975970|             1|
|    3980089|             1|
|    4614773|             2|
|    4672448|             2|
|    4816572|             1|
|    4876799|             1|
|    4991959|             1|
|    5011347|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates(subset=['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001007254|The Lion, the Wit...|
|0001053094|Angela's Ashes, 2...|
|0001712667|Oh, the Places Yo...|
|0001716107|Did I Ever Tell Y...|
|0001857010|The Lion, the Wit...|
|0002171856|The SAS Survival ...|
|0002245868|A Storm of Swords...|
|0002247283|Fool's Fate (Tawn...|
|000472383X|Collins Complete ...|
|0006167209|Sad Cypress (Agat...|
|0006168957|Death on the Nile...|
|0006174744|The Mysterious Af...|
|0006479235|The Martian Chron...|
|0006479898|A Song of Ice and...|
|0006480101|Royal Assassin (T...|
|0006486029|The Golden Fool: ...|
|0006496458|       Fourth Estate|
|0006531407|The SAS Survival ...|
|0006547745|From the Holy Mou...|
|0006550681|The God of Small ...|
+----------+--------------------+
only showing top 20 rows



In [15]:
products_df.describe()

DataFrame[summary: string, product_id: string, product_title: string]

In [18]:
products_df.summary().show()

+-------+--------------------+---------------------------------+
|summary|          product_id|                    product_title|
+-------+--------------------+---------------------------------+
|  count|               47736|                            47736|
|   mean|1.2376550348800678E9|               193061.32692307694|
| stddev|1.4792812168378117E9|               1386644.3942003178|
|    min|          0001007254|             "American Horror ...|
|    25%|        3.45501292E8|                             10.0|
|    50%|        7.52886215E8|                             25.0|
|    75%|       1.599290642E9|                           1984.0|
|    max|          B01N1Z4PKT|特撮リボルテック045 アイアンマ...|
+-------+--------------------+---------------------------------+



In [19]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RVOG49N0H1FB6|      10133|B004TACMZ8|     569741360| 2014-08-01|
| RNCMD6OLTP4HM|      19612|1846071224|     785505948| 2014-12-04|
| R4AUOBI8YC0R8|      19612|0375851569|     516548029| 2014-12-04|
|R1VSHIJ1RHIBTE|      19677|B0060SVG54|     302116447| 2015-07-16|
|R3JBLVALWSLCZD|      19999|B00EYQ6CVC|     368843515| 2014-02-08|
| RJ6GK77Y2NKCK|      20014|B00MC1CCAG|     663681053| 2014-11-11|
| R13W0E5EDX50J|      20152|B005D7FKWI|     182054491| 2013-10-04|
|R2L8UN6YSPY1CK|      20152|B007W9KQ9W|     251269761| 2013-10-04|
| RA2DFU68J832H|      20152|B005F3DFXG|     283218719| 2013-10-04|
|R191XKANDDV34N|      20152|B00EYXGOYK|     678332854| 2013-11-29|
| RD2NWWDF0S2B9|      20152|B00BRC6PYU|     715581077| 2014-05-21|
|R35SNJXI0Y9J1T|      20152|B00DZFMC4Y|     542832677| 2014-05

In [20]:
review_id_df.describe()

DataFrame[summary: string, review_id: string, customer_id: string, product_id: string, product_parent: string]

In [21]:
review_id_df.describe('review_date')

DataFrame[summary: string]

In [22]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RVOG49N0H1FB6|          5|            0|          0|   N|                Y|
| RNCMD6OLTP4HM|          5|            1|          1|   N|                Y|
| R4AUOBI8YC0R8|          5|            0|          0|   N|                Y|
|R1VSHIJ1RHIBTE|          5|            0|          0|   N|                Y|
|R3JBLVALWSLCZD|          5|            9|         14|   N|                Y|
| RJ6GK77Y2NKCK|          4|            6|          7|   N|                Y|
| R13W0E5EDX50J|          5|            0|          1|   N|                Y|
|R2L8UN6YSPY1CK|          3|            1|          3|   N|                Y|
| RA2DFU68J832H|          4|            1|          1|   N|                Y|
|R191XKANDDV34N|          4|            0|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [24]:
# Configure settings for RDS
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://challenge16.clajpbdznnel.us-east-2.rds.amazonaws.com:5432/challenge16"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [25]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [26]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [27]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [28]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)